In [1]:
import asyncio
import logging
import multiprocessing
from logging.handlers import QueueHandler, QueueListener
from multiprocessing import Process, Queue


def configure_worker_logging(log_queue: Queue):
    """Logging configuration inside worker process."""
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = QueueHandler(log_queue)
    root.handlers = [handler]  # Replace handlers with queue handler


def background_worker(log_queue: Queue,name:str):
    """Background process that logs from another process."""
    configure_worker_logging(log_queue)
    logger = logging.getLogger(f"worker[{name}]")

    logger.info("Worker started")

    # имитация фоновой обработки
    for i in range(3):
        logger.info(f"Worker iteration {i}")

    logger.info("Worker finished")

def main():
    # -----------------------
    # 1. Создаём очередь логов
    # -----------------------
    log_queue = multiprocessing.Queue()

    # -----------------------
    # 2. Настраиваем главный логгер FileHandler
    # -----------------------
    file_handler = logging.FileHandler("./tmp/multiprocessing_log.log", encoding="utf-8")
    file_handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "%(asctime)s | %(processName)s | %(name)s | %(levelname)s | %(message)s"
    )
    file_handler.setFormatter(formatter)

    # -----------------------
    # 3. QueueListener → один поток пишет в файл
    # -----------------------
    listener = QueueListener(log_queue, file_handler)
    listener.start()

    # -----------------------
    # 4. Инициализация базового логера
    # -----------------------
    configure_worker_logging(log_queue)
    logger = logging.getLogger("main")
    
    # -----------------------
    # 5. Запускаем рабочий процесс
    # -----------------------
    logger.info("=== Main process started ===")
    workers: list[Process] =[]
    for i in range(3):
        worker = Process(
            target=background_worker,
            args=(log_queue,i),
            name="BackgroundProcess",
            daemon=True
        )
        worker.start()
        workers.append( worker )
    logger.info("=== All workers started ===")

    # -----------------------
    # 7. Ждём worker
    # -----------------------
    logger.info("=== Waiting for workers to finish ===")
    for worker in workers:
        worker.join()

    # -----------------------
    # 8. Закрываем listener
    # -----------------------
    logger.info("=== wait for logger listener finished ===")
    listener.stop()


main()
